In [1]:
import pandas as pd
import subprocess

In [2]:
def generate_workspace():
    subprocess.run(["mkdir", "-p", "tmp_dir"])

In [3]:
import os


def cut_video(data:pd.Series, tmp_dir:str):
    """
    A function that cuts video according to pd.Series values.
    This function will execute following command using subprocess.
    ffmpeg -ss 00:01:00 -to 00:02:00  -i input.mp4 -c copy output.mp4
    
    :param data: a pandas.Series that represents the data of current video clip
    :param tmp_dir: a string that reprsents tmp_dir
    """
    start_str = data['start']
    end_str = data['end']
    country_name = data['name']
    save_name = data['save_name']
    out_file_name = os.path.join(tmp_dir, save_name)
    subprocess.run([
        "ffmpeg", "-ss", start_str, "-to",end_str, "-i", 
        "../OriginalVideo.mp4","-c", "copy", out_file_name, "-y"], 
        stdout=subprocess.DEVNULL,
        stderr=subprocess.STDOUT)

In [16]:
def change_video_speed(video_name:str, speed:float):
    """
    A function that changes video speed into designated speed.
    This will generate a video with %03ds.mp4.
    Example, if input.mp4 was processed, it will generate output.mp4.
    
    This function will execute following command using subprocess.
    ffmpeg -i Break0.mp4 -vf "setpts=(PTS-STARTPTS)/2.5" -af atempo=2.5 a.mp4

    :param video_name: a string that represents video's name to process.
    :param speed: a float value that represents how fast the video should go.
    """
    subprocess.run([
        "ffmpeg",
        "-i",
        video_name,
        "-vf",
        "setpts=(PTS-STARTPTS)/%f" % speed,
        "-af",
        "atempo=%f" %speed,
        video_name.replace(".mp4", "s.mp4"),
        "-y"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.STDOUT)

In [6]:
def add_caption(video_name:str, country_name:str, value:str, 
                font_dir:str, font_color:str, font_size:int,
                box_enabled:bool, box_opacity:float, box_color:str, 
                text_position:tuple):
    text = country_name + " : " + value
    subprocess.run([
        "ffmpeg",
        "-i",
        video_name,
        "-vf",
        "\"drawtext=fontfile=/%s:text='%s':fontcolor=white:fontsize=%d:box=%d:boxcolor=%s@0.5:boxborderw=5:x=(w-text_w)/2:y=(h-text_h)/2\"" % (font_dir, text, font_color, font_size, box_enabled, box_color, box_
        "-codec:a",
        "copy",
        video_name.replace("s.mp4", "f.mp4"),
        "-y"
    ])

In [9]:
if __name__ == "__main__":
    generate_workspace()
    tmp_dir = os.path.join(os.getcwd(), "tmp_dir")
    audio_dir = os.path.join(tmp_dir, "audio")
    data = pd.read_csv("../time_data.csv")
    cut_video(data.loc[0], tmp_dir)
    change_video_speed(os.path.join(tmp_dir, data.loc[0]['save_name']), 2)

In [1]:
from IPython.display import Video

Video("./tmp_dir/000s.mp4")

In [ ]:
add_caption("./tmp_dir/000s.mp4", "break", "109999", "../fonts/comic.ttf", )

In [4]:
Video("./tmp_dir/000f.mp4")